In [1]:
import os
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import pickle
import numpy as np

from ResTCN import ResTCN
from utils import get_dataloader

os.chdir(r"F:\RESTCN_CODE")

torch.manual_seed(0)
num_epochs = 25
batch_size = 6
lr = .001
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print("Device being used:", device, flush=True)


# dataloader = get_dataloader(batch_size,
#                             'train.csv',
#                             os.path.join(os.getcwd(), 'images_train'),
#                             'test.csv',
#                             os.path.join(os.getcwd(), 'images_test'))
dataloader = get_dataloader(batch_size,
                            'csv\\train.csv',
                            os.getcwd(),
                            'csv\\validation.csv',
                            os.getcwd())

dataset_sizes = {x: len(dataloader[x].dataset) for x in ['train','test']}
print(dataset_sizes, flush=True) # OUTPUT: {'train': 5482, 'test': 1784}


model = ResTCN().to(device)
model.load_state_dict(torch.load(r"resTCN_model.pth"))

# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=50, gamma=.1)

criterion = nn.CrossEntropyLoss().to(device)
softmax = nn.Softmax()

train_losses = []
val_losses = []
running_loss=0
for phase in ['test']:
        model.eval()
        running_loss = .0
        y_trues = np.empty([0])
        y_preds = np.empty([0])

        for inputs, labels in tqdm(dataloader[phase]):
            inputs = inputs.to(device)
            labels = labels.long().squeeze().to(device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)


            running_loss += loss.item() * inputs.size(0)
            try:
                preds = torch.max(softmax(outputs), dim=1)[1]  # Fix here
                # print('HI')
            except Exception as e:
                print (e)
                continue
            

            y_trues = np.append(y_trues, labels.data.cpu().numpy())
            y_preds = np.append(y_preds, preds.cpu())
            
            
        # if phase == 'train':
        #     scheduler.step()
        
        epoch_loss = running_loss / dataset_sizes[phase]
        val_losses.append(epoch_loss)

        print("[{}] Epoch: {}/{} Loss: {} LR: {}".format(
            phase, 0 + 1, num_epochs, epoch_loss, scheduler.get_last_lr()), flush=True)
        print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))
        print('\naccuracy\t' + str(accuracy_score(y_trues, y_preds)))



c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device being used: cuda
{'train': 5482, 'test': 1720}


c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0

[test] Epoch: 1/25 Loss: 1.494581714385124 LR: [0.001]

confusion matrix
[[  0   0  17   6]
 [  0   6  94  60]
 [  0  22 454 436]
 [  0  15 212 398]]

accuracy	0.49883720930232556


In [2]:
import os
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import pickle
import numpy as np

from ResTCN import ResTCN
from utils import get_dataloader

os.chdir(r"F:\RESTCN_CODE")

torch.manual_seed(0)
num_epochs = 25
batch_size = 6
lr = .001
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print("Device being used:", device, flush=True)


# dataloader = get_dataloader(batch_size,
#                             'train.csv',
#                             os.path.join(os.getcwd(), 'images_train'),
#                             'test.csv',
#                             os.path.join(os.getcwd(), 'images_test'))
dataloader = get_dataloader(batch_size,
                            'csv\\train.csv',
                            os.getcwd(),
                            'csv\\validation.csv',
                            os.getcwd())

dataset_sizes = {x: len(dataloader[x].dataset) for x in ['train','test']}
print(dataset_sizes, flush=True) # OUTPUT: {'train': 5482, 'test': 1784}


model = ResTCN().to(device)
model.load_state_dict(torch.load(r"resTCN_model_weighted.pth"))

# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=50, gamma=.1)

criterion = nn.CrossEntropyLoss().to(device)
softmax = nn.Softmax()

train_losses = []
val_losses = []
running_loss=0
for phase in ['test']:
        model.eval()
        running_loss = .0
        y_trues = np.empty([0])
        y_preds = np.empty([0])

        for inputs, labels in tqdm(dataloader[phase]):
            inputs = inputs.to(device)
            labels = labels.long().squeeze().to(device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)


            running_loss += loss.item() * inputs.size(0)
            try:
                preds = torch.max(softmax(outputs), dim=1)[1]  # Fix here
                # print('HI')
            except Exception as e:
                print (e)
                continue
            

            y_trues = np.append(y_trues, labels.data.cpu().numpy())
            y_preds = np.append(y_preds, preds.cpu())
            
            
        # if phase == 'train':
        #     scheduler.step()
        
        epoch_loss = running_loss / dataset_sizes[phase]
        val_losses.append(epoch_loss)

        print("[{}] Epoch: {}/{} Loss: {} LR: {}".format(
            phase, 0 + 1, num_epochs, epoch_loss, scheduler.get_last_lr()), flush=True)
        print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))
        print('\naccuracy\t' + str(accuracy_score(y_trues, y_preds)))



Device being used: cuda
{'train': 5482, 'test': 1720}


c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0

[test] Epoch: 1/25 Loss: 1.2584605281782704 LR: [0.001]

confusion matrix
[[  1   1  12   9]
 [  2   6  83  69]
 [  5  45 391 471]
 [  6  40 216 363]]

accuracy	0.4424418604651163


In [3]:
import os
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import pickle
import numpy as np

from ResTCN import ResTCN
from utils import get_dataloader

os.chdir(r"F:\RESTCN_CODE")

torch.manual_seed(0)
num_epochs = 25
batch_size = 6
lr = .001
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print("Device being used:", device, flush=True)


# dataloader = get_dataloader(batch_size,
#                             'train.csv',
#                             os.path.join(os.getcwd(), 'images_train'),
#                             'test.csv',
#                             os.path.join(os.getcwd(), 'images_test'))
dataloader = get_dataloader(batch_size,
                            'csv\\train.csv',
                            os.getcwd(),
                            'csv\\validation.csv',
                            os.getcwd())

dataset_sizes = {x: len(dataloader[x].dataset) for x in ['train','test']}
print(dataset_sizes, flush=True) # OUTPUT: {'train': 5482, 'test': 1784}


model = ResTCN().to(device)
model.load_state_dict(torch.load(r"resTCN_model_aug_weighted.pth"))

# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=50, gamma=.1)

criterion = nn.CrossEntropyLoss().to(device)
softmax = nn.Softmax()

train_losses = []
val_losses = []
running_loss=0
for phase in ['test']:
        model.eval()
        running_loss = .0
        y_trues = np.empty([0])
        y_preds = np.empty([0])

        for inputs, labels in tqdm(dataloader[phase]):
            inputs = inputs.to(device)
            labels = labels.long().squeeze().to(device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)


            running_loss += loss.item() * inputs.size(0)
            try:
                preds = torch.max(softmax(outputs), dim=1)[1]  # Fix here
                # print('HI')
            except Exception as e:
                print (e)
                continue
            

            y_trues = np.append(y_trues, labels.data.cpu().numpy())
            y_preds = np.append(y_preds, preds.cpu())
            
            
        # if phase == 'train':
        #     scheduler.step()
        
        epoch_loss = running_loss / dataset_sizes[phase]
        val_losses.append(epoch_loss)

        print("[{}] Epoch: {}/{} Loss: {} LR: {}".format(
            phase, 0 + 1, num_epochs, epoch_loss, scheduler.get_last_lr()), flush=True)
        print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))
        print('\naccuracy\t' + str(accuracy_score(y_trues, y_preds)))



Device being used: cuda
{'train': 5482, 'test': 1720}


c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0

[test] Epoch: 1/25 Loss: nan LR: [0.001]

confusion matrix
[[ 23   0   0   0]
 [160   0   0   0]
 [912   0   0   0]
 [625   0   0   0]]

accuracy	0.013372093023255814


In [4]:
import os
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import pickle
import numpy as np

from ResTCN import ResTCN
from utils import get_dataloader

os.chdir(r"F:\RESTCN_CODE")

torch.manual_seed(0)
num_epochs = 25
batch_size = 6
lr = .001
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print("Device being used:", device, flush=True)


# dataloader = get_dataloader(batch_size,
#                             'train.csv',
#                             os.path.join(os.getcwd(), 'images_train'),
#                             'test.csv',
#                             os.path.join(os.getcwd(), 'images_test'))
dataloader = get_dataloader(batch_size,
                            'csv\\train.csv',
                            os.getcwd(),
                            'csv\\validation.csv',
                            os.getcwd())

dataset_sizes = {x: len(dataloader[x].dataset) for x in ['train','test']}
print(dataset_sizes, flush=True) # OUTPUT: {'train': 5482, 'test': 1784}


model = ResTCN().to(device)
model.load_state_dict(torch.load(r"resTCN_mod_model_weighted.pth"))

# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=50, gamma=.1)

criterion = nn.CrossEntropyLoss().to(device)
softmax = nn.Softmax()

train_losses = []
val_losses = []
running_loss=0
for phase in ['test']:
        model.eval()
        running_loss = .0
        y_trues = np.empty([0])
        y_preds = np.empty([0])

        for inputs, labels in tqdm(dataloader[phase]):
            inputs = inputs.to(device)
            labels = labels.long().squeeze().to(device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)


            running_loss += loss.item() * inputs.size(0)
            try:
                preds = torch.max(softmax(outputs), dim=1)[1]  # Fix here
                # print('HI')
            except Exception as e:
                print (e)
                continue
            

            y_trues = np.append(y_trues, labels.data.cpu().numpy())
            y_preds = np.append(y_preds, preds.cpu())
            
            
        # if phase == 'train':
        #     scheduler.step()
        
        epoch_loss = running_loss / dataset_sizes[phase]
        val_losses.append(epoch_loss)

        print("[{}] Epoch: {}/{} Loss: {} LR: {}".format(
            phase, 0 + 1, num_epochs, epoch_loss, scheduler.get_last_lr()), flush=True)
        print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))
        print('\naccuracy\t' + str(accuracy_score(y_trues, y_preds)))



Device being used: cuda
{'train': 5482, 'test': 1720}


c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0

[test] Epoch: 1/25 Loss: 1.6277855031365571 LR: [0.001]

confusion matrix
[[  0   1  19   3]
 [  0   0  93  67]
 [  0  15 466 431]
 [  0   9 217 399]]

accuracy	0.502906976744186


In [5]:
import os
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch
from torch import nn, optim
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
import pickle
import numpy as np

from ResTCN import ResTCN
from utils import get_dataloader

os.chdir(r"F:\RESTCN_CODE")

torch.manual_seed(0)
num_epochs = 25
batch_size = 6
lr = .001
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print("Device being used:", device, flush=True)


# dataloader = get_dataloader(batch_size,
#                             'train.csv',
#                             os.path.join(os.getcwd(), 'images_train'),
#                             'test.csv',
#                             os.path.join(os.getcwd(), 'images_test'))
dataloader = get_dataloader(batch_size,
                            'csv\\train.csv',
                            os.getcwd(),
                            'csv\\validation.csv',
                            os.getcwd())

dataset_sizes = {x: len(dataloader[x].dataset) for x in ['train','test']}
print(dataset_sizes, flush=True) # OUTPUT: {'train': 5482, 'test': 1784}


model = ResTCN().to(device)
model.load_state_dict(torch.load(r"resTCN_mod_model_weighted-1.pth"))

# optimizer = optim.Adam(model.parameters(), lr=lr)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = StepLR(optimizer, step_size=50, gamma=.1)

criterion = nn.CrossEntropyLoss().to(device)
softmax = nn.Softmax()

train_losses = []
val_losses = []
running_loss=0
for phase in ['test']:
        model.eval()
        running_loss = .0
        y_trues = np.empty([0])
        y_preds = np.empty([0])

        for inputs, labels in tqdm(dataloader[phase]):
            inputs = inputs.to(device)
            labels = labels.long().squeeze().to(device)

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs).squeeze()
                loss = criterion(outputs, labels)


            running_loss += loss.item() * inputs.size(0)
            try:
                preds = torch.max(softmax(outputs), dim=1)[1]  # Fix here
                # print('HI')
            except Exception as e:
                print (e)
                continue
            

            y_trues = np.append(y_trues, labels.data.cpu().numpy())
            y_preds = np.append(y_preds, preds.cpu())
            
            
        # if phase == 'train':
        #     scheduler.step()
        
        epoch_loss = running_loss / dataset_sizes[phase]
        val_losses.append(epoch_loss)

        print("[{}] Epoch: {}/{} Loss: {} LR: {}".format(
            phase, 0 + 1, num_epochs, epoch_loss, scheduler.get_last_lr()), flush=True)
        print('\nconfusion matrix\n' + str(confusion_matrix(y_trues, y_preds)))
        print('\naccuracy\t' + str(accuracy_score(y_trues, y_preds)))



Device being used: cuda
{'train': 5482, 'test': 1720}


c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Manvendra Nema\anaconda3\envs\vercil\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0

[test] Epoch: 1/25 Loss: 1.6461322551203328 LR: [0.001]

confusion matrix
[[  0   0  11  12]
 [  0   0  60 100]
 [  0   0 340 572]
 [  0   0 174 451]]

accuracy	0.45988372093023255
